In [1]:
import nltk
from nltk.stem import PorterStemmer
#nltk.download()

In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

In [3]:
stop_words = set(['a','about', 'above','after','again', 'against','ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as',\
 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd',\
 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from',\
 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here',\
 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into','is', 'isn', "isn't", 'it', "it's", 'its', 'itself',\
 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't",\
 'no', 'nor', 'not','now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over',\
 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such',\
 't', 'than', 'that', "that'll",'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this',\
 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren',\
 "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn',\
 "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'])

In [4]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [5]:
corpus = sc.textFile("WikipediaPagesOneDocPerLine1000LinesSmall.txt")
# fucntion to count number of occurences of eacch word in each documents and 
# calculate term frequences
def buildArray (listOfIndices):
    returnVal = np.zeros (20000)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum (returnVal)
    returnVal = np.divide (returnVal, mysum)
    return returnVal

# function to get array whose element is 0 if the tf is 0, otherwise 1
def zeroOneArray(listOfTFs):
    returnVal = np.zeros(20000)
    for i in range(len(listOfTFs)):
        if listOfTFs[i] != 0.0:
            returnVal[i] = 1.0
    return returnVal

import sys
from operator import add
import re

import numpy as np

from operator import add
# create number of docs
numberOfDocs = corpus.count()

# filter out lines where do not have id and url field
validLines = corpus.filter(lambda x : 'id' in x and 'url=' in x)

# map to key value pairs whose key is id and value is document body..
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

# create regex object, make document all lowercase and split by " "
# [('431949', ['black', 'people', 'and', 'mormonismwhile'...
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))


In [6]:
keyAndListOfWords.take(5)

[('431949',
  ['black',
   'people',
   'and',
   'mormonismwhile',
   'at',
   'least',
   'two',
   'black',
   'men',
   'held',
   'the',
   'priesthood',
   'in',
   'the',
   'early',
   'church',
   'from',
   'the',
   'mid',
   's',
   'until',
   'the',
   'church',
   'of',
   'jesus',
   'christ',
   'of',
   'latter',
   'day',
   'saints',
   'lds',
   'church',
   'had',
   'a',
   'policy',
   'which',
   'prevented',
   'most',
   'men',
   'of',
   'black',
   'african',
   'descent',
   'from',
   'being',
   'ordained',
   'to',
   'the',
   'church',
   's',
   'lay',
   'priesthood',
   'under',
   'the',
   'temple',
   'and',
   'priesthood',
   'restrictions',
   'before',
   'black',
   'members',
   'of',
   'african',
   'descent',
   'could',
   'not',
   'receive',
   'the',
   'priesthood',
   'or',
   'participate',
   'in',
   'temple',
   'ordinances',
   'besides',
   'baptisms',
   'for',
   'the',
   'dead',
   'for',
   'a',
   'time',
   'in',
   

In [7]:
ps = PorterStemmer()

In [8]:
# flat map each word to key value pair like ("word", 1)
# ex [('black', 1), ('people', 1), ('and', 1), ('mormonismwhile', 1), ('at', 1)......
allWords =keyAndListOfWords.flatMap(lambda x: x[1])\
                            .filter(lambda x: x not in stop_words)\
                            .map(lambda x: (x, 1))

In [9]:
# aggregate to cout
# [('at', 4810), ('two', 1848), ('priesthood', 78), ('in', 27758), ('mid', 159).....
allCounts = allWords.reduceByKey(add)

# get top 20000 words
topWords = allCounts.top(20000, key=lambda x: x[1])

# get key value pairs whose key is word and value is position rank of within top 20000
twentyK = sc.parallelize(range(20000))
dictionary = twentyK.map (lambda x : (topWords[x][0], x))

# get key value pairs whose keys are all words in documnt and values are ids
# (word, id)
allWords = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

# join rdd to get key value pairs like ("word", (rank, id))
allDictionaryWords = dictionary.join(allWords)

# map to (id, rank)
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

# group by id 
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

# aggregate to numpy array of ranks and concatinate string 
forCSV= allDictionaryWordsInEachDoc.mapValues(lambda x: np.array( list(x)))



In [10]:
###### From here task2

# calculate term frequences for each word in each document
tfs = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))

# calculate array whose elememts are 0 or 1
zeroOrOne = tfs.map(lambda x: (x[0], zeroOneArray(x[1])))

# calculate word in document occurences within the corpuse
dfs = zeroOrOne.reduce (lambda x1, x2:(("", np.add (x1[1], x2[1]))))[1]

# calculate inverse document frequency 
multiplier = np.full (20000, numberOfDocs)
idfs = np.log (np.divide(numberOfDocs, dfs))

#  calculate tfidf
tfidfs = tfs.map (lambda x: (x[0], np.multiply (x[1], idfs)))

In [11]:
idfs

array([0.44785082, 0.51082562, 0.61803971, 0.80073239, 0.63676685,
       0.82553637, 1.05268336, 0.74023879, 0.91879386, 0.95972029,
       1.174414  , 0.92886951, 1.10866262, 0.94417594, 0.99695863,
       1.45671683, 1.11779511, 1.28735441, 0.98082925, 1.43969514,
       1.23787436, 1.35479569, 1.00239343, 1.05268336, 2.45340798,
       1.41469384, 1.0441241 , 0.94933059, 0.99425227, 1.39432653,
       1.65025991, 1.42711636, 1.14256418, 1.41881755, 1.08470938,
       1.52785793, 1.70925825, 1.13010296, 2.02495336, 1.7837913 ,
       1.06421086, 1.11779511, 1.30563646, 1.24827306, 1.15518264,
       1.2552661 , 1.60943791, 1.72597173, 1.81400508, 1.44816976,
       1.30563646, 1.35092722, 1.70374859, 2.34340709, 1.71479843,
       1.54177926, 2.16282315, 1.26940061, 1.29462717, 1.67664666,
       1.91054301, 2.3330443 , 1.80788885, 2.45340798, 1.44392347,
       1.41469384, 1.91732269, 1.65548185, 1.97328135, 1.2552661 ,
       2.15416509, 1.82015894, 1.28735441, 1.39836694, 1.50507

In [12]:
def getPrediction(textInput, k):
        
        # make rdd for document 
        target = sc.parallelize(("", textInput))
        # (word, 1)
        wordsArray = target.flatMap(lambda x: ((i, 1) for i in regex.sub(' ', x).lower().split()))
        # dictionary(word, dicPosition) join with wordArray(word, 1) to make (word ,(dicPosition, 1))
        dictionaryWords = dictionary.join(wordsArray)
        # groupby to count number of each word in the new document
        # (1, [dicPosi, dicPosi])      
        allDictionaryWordsInThatDoc = dictionaryWords.map(lambda x: (x[1][1], x[1][0]))\
                                                .groupByKey()
                                                #.mapValues(lambda x: np.array(list(x)))
        # calculate term frequences for each word in each document
        targetTfs = buildArray(allDictionaryWordsInThatDoc.top(1)[0][1])
        
        # calculate array whose elememts are 0 or 1
        # targetZeroOrOne = zeroOneArray(targetTfs)
        # calculate document occurences within the corpuse
        # targetDf = np.sum(targetZeroOrOne)

        # targetIdfs = np.log(numberOfDocs / targetDf)

        targetTfidfs = np.multiply(targetTfs, idfs)
        # calculate cosine similality between textnput and all docs
        distances = tfidfs.map(lambda x: (x[0], np.dot(x[1], targetTfidfs)))
        # select top k similarity (docID, similaliry)
        topK = distances.top(k, lambda x: x[1])
        # count how many time each docID appear in the top k similar docID
        # docIDRepresented = sc.parallelize(topK).map (lambda x : (x[0], 1))
        # numTimes = docIDRepresented.reduceByKey(add)
        # take top 1
        # numTimes.top(1, lambda x: x[1])
        return topK

In [13]:
print(getPrediction('Sport Basketball Volleyball Soccer', 1))

[('432240', 0.328838508692316)]


In [14]:
print(getPrediction("How many goals Vancouver score last year?", 30))

[('454889', 0.1614193434553638), ('418348', 0.12243361158460944), ('455834', 0.06752601394638107), ('429246', 0.042908939399527085), ('418347', 0.040538443281177804), ('429279', 0.03593512405241609), ('418346', 0.032854959199157034), ('431971', 0.02791545789058566), ('433651', 0.02560693364606556), ('433602', 0.02560693364606556), ('433633', 0.02560693364606556), ('433675', 0.02560693364606556), ('454907', 0.022551933895175024), ('429092', 0.016470274018518645), ('432141', 0.01463253351203746), ('418609', 0.014120873090296122), ('455202', 0.011708930427061084), ('432109', 0.011362077089799005), ('431975', 0.011353791674346714), ('429261', 0.011274717865146137), ('418387', 0.010271411597489302), ('434005', 0.009755022341358308), ('428953', 0.00974430100869649), ('432236', 0.009575107141696808), ('431952', 0.009345360554683128), ('455203', 0.009295797142157508), ('432283', 0.008299676015852991), ('432364', 0.007910153181298857), ('418595', 0.007664660411067242), ('432138', 0.007639429523